In [1]:
# pip install shap

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor

from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, OrdinalEncoder
# from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline 
from sklearn.compose import ColumnTransformer

from sklearn.metrics import (
        classification_report, confusion_matrix, r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, accuracy_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve
) 

from sklearn.inspection import PartialDependenceDisplay, permutation_importance, partial_dependence

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold
from xgboost import XGBClassifier, XGBRegressor, plot_importance
from sklearn.model_selection import cross_val_score

In [5]:
from category_encoders import TargetEncoder 
from sklearn.feature_selection import SelectFromModel

In [6]:
# from sklearn.datasets import make_classification

In [7]:
from imblearn.over_sampling import SMOTE
from collections import Counter

In [8]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer 

In [9]:
from feature_engine.imputation import CategoricalImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor

In [10]:
from xgbimputer import XGBImputer

In [11]:
import tqdm as notebook_tqdm

In [12]:
import shap

In [13]:
In2019_withFresh_Y_Pred = pd.read_csv('../data/In2019_withFresh_Y_Pred_TimeIntervalClass.csv') 

In [14]:
In2019_withFresh_Y_Pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112019 entries, 0 to 112018
Data columns (total 55 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0.2                    112019 non-null  int64  
 1   Unnamed: 0.1                    112019 non-null  int64  
 2   Unnamed: 0                      112019 non-null  int64  
 3   RecordID                        112019 non-null  int64  
 4   ESS_Id                          112019 non-null  int64  
 5   EventTimeStamp                  112019 non-null  object 
 6   eventDescription                108067 non-null  object 
 7   actionDescription               0 non-null       float64
 8   ecuSoftwareVersion              49117 non-null   object 
 9   ecuSerialNumber                 46794 non-null   object 
 10  ecuModel                        104416 non-null  object 
 11  ecuMake                         104416 non-null  object 
 12  ecuSource       

**Goal:** Predict the rank based on the total shots.

In [15]:
In2019_withFresh_Y_Pred['spn_fmi'] = In2019_withFresh_Y_Pred['spn_fmi'].str.replace('.0', '')

In [16]:
def calculate_equipment_savings(data_path):
    """
    Calculate potential savings or losses based on equipment data analysis.
    
    Conditions:
    1. If SPN 5246 exists in an EquipID_Index group AND Predicted_Time_Interval_Class contains 1: $4000 saved
    2. If SPN 5246 doesn't exist in an EquipID_Index group AND Predicted_Time_Interval_Class contains 1: $500 lost
    
    Args:
        data_path (str): Path to the data file (CSV expected)
        
    Returns:
        float: Net savings (positive) or losses (negative) across all equipment
    """
    # Load the data
    print(f"Loading data from {data_path}...")
    df = pd.read_csv(data_path)
    
    # Display data info for verification
    print(f"Data loaded. Shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Number of unique EquipID_Index: {df['EquipID_Index'].nunique()}")
    
    # Initialize variables to track total savings/losses
    total_savings = 0
    total_losses = 0
    
    # Create a results dataframe to store savings/losses by equipment
    results = []
    
    # Group by EquipID_Index
    EquipID_Index_Group = df.groupby('EquipID_Index')
    
    # Process each equipment group
    for EquipID_Index, group in EquipID_Index_Group:
        # Check if SPN 5246 exists in this group
        has_spn_5246 = (group['spn'] == 5246).any()
        
        # Check if Predicted_Time_Interval_Class contains 1
        has_time_interval_1 = (group['Predicted_Time_Interval_Class'] == 1).any()
        
        savings = 0
        losses = 0
        
        # Apply the business rules
        if has_spn_5246 and has_time_interval_1:
            # Condition 1: SPN 5246 exists and Predicted_Time_Interval_Class contains 1
            savings = 4000
            total_savings += savings
        elif not has_spn_5246 and has_time_interval_1:
            # Condition 2: SPN 5246 doesn't exist and Predicted_Time_Interval_Class contains 1
            losses = 500
            total_losses += losses
        
        # Store the results for this equipment
        results.append({
            'EquipID_Index': EquipID_Index,
            'Has_SPN_5246': has_spn_5246,
            'Has_Time_Interval_1': has_time_interval_1,
            'Savings': savings,
            'Losses': losses,
            'Net': savings - losses
        })
    
    # Create a results dataframe
    results_df = pd.DataFrame(results)
    
    # Calculate net savings across all equipment
    net_saving_or_loss = total_savings - total_losses
    
    # Display summary
    print("\nSummary of Results:")
    print(f"Total Count of EquipmentID_Index by Full Derates or Lack Thereof: {len(results)}")
    print(f"Number of Full Derate True Positive Predictions with Savings: {(results_df['Savings'] > 0).sum()}")
    print(f"Number of Full Derate False Positive Predictions with Losses: {(results_df['Losses'] > 0).sum()}")
    print(f"Total Savings: ${total_savings:,.2f}")
    print(f"Total Losses: ${total_losses:,.2f}")
    print(f"Net Savings or Losses: ${net_saving_or_loss:,.2f}")
    
    # Display a few sample results
    print("\nSample equipment results:")
    print(results_df.head(10))
    
    # Save the detailed results to CSV
    results_df.to_csv('../data/equipment_savings_analysis_byTimeIntervalClass.csv', index=False)
    print("Detailed results saved to 'equipment_savings_analysis.csv'")
    
    return net_saving_or_loss

In [17]:
if __name__ == "__main__":
    # Replace with the actual path to your data file
    data_file = "../data/In2019_withFresh_Y_Pred_TimeIntervalClass.csv"
    
    try:
        net_result = calculate_equipment_savings(data_file)
        
        if net_result > 0:
            print(f"\nOverall result: Net savings of ${net_result:,.2f}")
        else:
            print(f"\nOverall result: Net loss of ${-net_result:,.2f}")
    
    except Exception as e:
        print(f"Error processing data: {e}")

Loading data from ../data/In2019_withFresh_Y_Pred_TimeIntervalClass.csv...
Data loaded. Shape: (112019, 55)
Columns: ['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'RecordID', 'ESS_Id', 'EventTimeStamp', 'eventDescription', 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'spn', 'fmi', 'active', 'activeTransitionCount', 'faultValue', 'EquipmentID', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp', 'FaultId', 'AcceleratorPedal', 'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus', 'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure', 'spn_fmi', 'is_fullderate', 'is_fullderate_group', 'EquipID_Index', 'time_to_next_SPN5246', '